# Abnormality detection of danish address valuations
**02807 Computational tools for data science**

All code can be found in our GitHub repository: [https://github.com/PBonvang/02807-Project](https://github.com/PBonvang/02807-Project)

Data should now be available at [https://drive.proton.me/urls/89G58EHFV4#8H7CXNedYN7k](https://drive.proton.me/urls/89G58EHFV4#8H7CXNedYN7k). 

You may contact s214606@dtu.dk if the link is locked or s214658@student.dtu.dk if GitHub repository is unavailable

### Dependencies and imports

In [ ]:
# Dependencies and imports
from pathlib import Path
import sys, os, glob, json, csv, re, pickle, warnings
from collections import defaultdict
from tqdm import tqdm


import pandas as pd
from pandas import DataFrame, concat
import numpy as np
import matplotlib.pyplot as plt

from requests_ratelimiter import LimiterSession

from analysis_config import PREDICTION_VAR
from config import LIST_IDENTIFIER,BBR_DATA_PATH, DATA_FOLDER

from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from scipy.stats import norm
from sklearn.metrics import confusion_matrix

## Configs

Most cells in this notebook refer to files in the GitHub repo e.g 
```py
# utils/preprocessing
```
represents our preprocessing functions file in utils directory. 

In [ ]:
# config
DATA_FOLDER = Path('Data')

BBR_DATA_FILENAME = 'BBR_FULL.json'
BBR_DATA_PATH = DATA_FOLDER / BBR_DATA_FILENAME
LIST_IDENTIFIER = 'List":'

In [ ]:
# analysis_config
DATA_FOLDER = 'data'
PREDICTION_VAR = 'totalAddressValue'

# Getting data
Reminder that BBR and DAR are either downloaded from datafordeleren.dk or their FTP server

### Scraping property valuations ("VUR")

In [ ]:
# scrape . Run from command line
session = LimiterSession(per_second=10)

def property_assessment(addressID: str, verbose=False):
    url = "https://api-fs.vurderingsportalen.dk/preliminaryproperties/_search"

    query = {
      "query": {
        "bool": {
          "must": [
            {
              "match_phrase": {
                "adgangsAdresseID": f"{addressID}"
              }
            }
          ],
          "filter": [
            {
              "bool": {
                "should": []
              }
            },
            {
              "match": {
                "documentType": 4
              }
            }
          ]
        }
      }
    }

    headers = {
        "Content-Type": "application/json",
    }
    
    response = session.post(url, json=query, headers=headers)

    if verbose:
        if response.status_code == 200:
            print("Success:")
            print(response.json())
        else:
            print("Failed to fetch data:")
            print("Status code:", response.status_code)
            print("Response:", response.text)
        
    return response.json()

def scrape(idx_start:int, idx_end:int, DAR_PATH:str='DAR_joined.csv', debug=False):
    """
    Can be run with "python <path to scrape.py> <idx_start> <idx_end> <DAR_PATH> <debug>"
    
    Args:
        idx_start (int): Start index of scraping. Must be greater than or equal to 0
        idx_end (int): End index of scraping. Must be less or equal to 2259957
        DAR_PATH (str): Path to "DAR_joined.csv"
        debug (bool, optional): Enables verbose and checks if 3 debug samples or scraped (index 0 to 2). Defaults to False.
    """
    columns = {'id':int,'adgangsAdresseID':str,
               'vurderingsaar':int,'propertyValue':float,
               'groundValue':float,'propertyTax':float,
               'groundTax':float,'totalAddressTax':float}

    debug_samples = ['0a3f507c-036b-32b8-e044-0003ba298018','2edda757-9218-1605-e044-0003ba298018','0a3f5083-e355-32b8-e044-0003ba298018'] #Middle one is invalid

    output_file = f'VUR_{idx_start}_to_{idx_end}.csv'
    
    pd.DataFrame(columns=columns.keys()).to_csv(output_file, index=False, sep=';')

    if debug:
        addressIDs = debug_samples
    else:
        addressIDs = pd.read_csv(DAR_PATH, delimiter=';').husnummer_id.iloc[idx_start:idx_end]

    for id in tqdm(addressIDs, desc='Scraping in progress', total=len(addressIDs)):
        response = property_assessment(id, verbose=debug)
        try:
            source = response['hits']['hits'][0]['_source']
            taxCalculations = source.pop('taxCalculation')
            source.update({k:re.sub("[^0-9]", "", v) for k, v in taxCalculations.items()})
            pd.DataFrame({key:source[key] for key in columns.keys()},index=[0]).to_csv(output_file, mode='a',index=False, header=False, sep=';')
        except IndexError:
            pass
        
        except KeyError as error:
            
            if error.args[0] == 'taxCalculation': # Basically fill tax columns with NaNs
              for k in columns.keys():
                pattern = re.compile(r'tax', re.IGNORECASE)
                if pattern.search(k):
                  source[k] = np.nan
            pd.DataFrame({key:source[key] for key in columns.keys()},index=[0]).to_csv(output_file, mode='a',index=False, header=False, sep=';')
                  
        
    VUR_edit = pd.read_csv(output_file, delimiter=';')
    for col, dtype in columns.items():
        VUR_edit[col] = VUR_edit[col].astype(dtype)
    VUR_edit.to_csv(output_file,index=False, header=True, sep=';')

scrape(*map(int,sys.argv[1:3]),*sys.argv[3:5]) # Run it CL (easier like this for HPC and other machines) e.g scrape(1123328,1123332,r'<path_to>\DAR_joined.csv')

Was run on HPC with
```bash
data/submit.sh
```
partition example:

In [ ]:
#!/bin/bash
#BSUB -J VUR
#BSUB -o VUR_LOG_%J.out  
#BSUB -q hpc
#BSUB -W 70:00
#BSUB -R "rusage[mem=8GB]"
#BSUB -n 12
#BSUB -R "span[hosts=1]"
#BSUB -N

set -e
module load python3/3.9.11
source ../../.venv/bin/activate
python3 -m pip install -r ../../requirements.txt
python3 scrape.py 2000000 2259957

# Preprocessing

### Utils


In [ ]:
# utils/preprocessing

from config import LIST_IDENTIFIER

def get_attribute_from_json_line(line: str)->str:
    return line.split(':')[0][1:-1]

def clean_file(data_path: Path, output_path: Path) -> None:
    with tqdm(total=data_path.stat().st_size) as pbar:
        with data_path.open(encoding='utf-8') as data_file:
            with output_path.open('w', encoding='utf-8') as output_file:
                while line := data_file.readline():
                    if line.strip() == '': continue

                    output_file.write(line)

def get_file_metadata(data_path: Path, save_path: Path = None)->dict:
    file_metadata = dict()

    with tqdm(total=data_path.stat().st_size) as pbar:
        with data_path.open(encoding='utf-8') as file:
            sample = {}
            determine_attributes = False

            i = 0
            list_name = ''
            while line := file.readline():
                if LIST_IDENTIFIER in line:
                    if list_name in file_metadata: 
                        file_metadata[list_name]['end_line'] = i-2
                        file_metadata[list_name]['attributes'] = list(sample.keys())
                        file_metadata[list_name]['sample'] = sample

                    list_name = get_attribute_from_json_line(line)
                    file_metadata[list_name] = {'start_line':i}

                    sample = {}
                    determine_attributes = True
                
                elif determine_attributes and '}' in line:
                    determine_attributes = False
                
                elif determine_attributes and ':' in line:
                    attribute, value = get_json_key_value_pair(line)
                    sample[attribute] = value
                    
                i += 1
                if not i % 1000:
                    pbar.update(file.tell() - pbar.n)

            if list_name in file_metadata: 
                file_metadata[list_name]['end_line'] = i-2
                file_metadata[list_name]['attributes'] = list(sample.keys())
                file_metadata[list_name]['sample'] = sample
                
    if save_path is not None:
        with save_path.open('w', encoding='utf-8') as outfile:
            json.dump(file_metadata, outfile)

    return file_metadata

def get_json_key_value_pair(json_line: str) -> tuple[str]:
    attribute, value = list(
        map(str.strip, json_line\
            .replace('"','')\
            .replace(',\n','')\
            .replace('\n','')\
            .split(':',1))
    )
    return attribute, value

def json_list_to_csv(json_file: Path,
                     list_metadata: dict,
                     output_file: Path,
                     attributes: list[str] = None) -> None:
    
    if attributes is None:
        attributes = list_metadata['attributes']

    with tqdm(total=list_metadata['end_line']) as pbar:
        with json_file.open(encoding='utf-8') as file:
            with output_file.open('w', encoding='utf-8',newline='') as output_file:
                csvwriter = csv.DictWriter(output_file, fieldnames=attributes, delimiter=";")
                csvwriter.writeheader()

                data_point = {}
                for i in range(list_metadata['end_line']):
                    line = file.readline()

                    if not i % 1000:
                        pbar.update(1000)

                    if i <= list_metadata['start_line']: # Skip to list
                        continue
                    
                    if ':' in line:
                        attribute, value = get_json_key_value_pair(line)
                        if attribute in attributes:
                            data_point[attribute] = value

                    if '}' in line and len(data_point.keys()) > 0:
                        csvwriter.writerow(data_point)
                        data_point = {}

### Bygnings- og Boligregistret (BBR)

In [ ]:
# extract_building_list
#sys.path.append(os.getcwd())

BUILDING_LIST_LINE_NUMBER = 55_911_629

def extract_specific_sequence_from_json_file(outfile_name: str, start_index: int = 0, start_character: str = '[', end_character: str = ']') -> None:
    i = 0
    outfile_path = DATA_FOLDER / outfile_name
    with tqdm(total=BBR_DATA_PATH.stat().st_size) as pbar:
        with BBR_DATA_PATH.open(encoding='utf-8') as file:
            with outfile_path.open('w', encoding='utf-8') as outfile:
                while line := file.readline():
                    i += 1
                    if not i % 1000:
                        pbar.update(file.tell() - pbar.n)
                    if i < start_index:
                        continue
                    outfile.write(line)

                    last_character_of_line = line.strip()[-1]
                    if last_character_of_line == end_character:
                        break


extract_specific_sequence_from_json_file('BuildingListBBR_FULL.json', BUILDING_LIST_LINE_NUMBER)

In [ ]:
# get_indices_of_lists

#sys.path.append(os.getcwd())

LIST_IDENTIFIER = 'List":'

def get_attribute_from_json_line(line: str)->str:
    return line.split(':')[0][1:-1]

def get_indices_of_lists()->dict:
    i = 0
    list_indices = dict()
    with tqdm(total=BBR_DATA_PATH.stat().st_size) as pbar:
        with BBR_DATA_PATH.open(encoding='utf-8') as file:
            while line := file.readline():
                if LIST_IDENTIFIER in line:
                    attribute = get_attribute_from_json_line(line)
                    list_indices[attribute] = i
                    
                i += 1
                if not i % 1000:
                    pbar.update(file.tell() - pbar.n)

    return list_indices

def save_indices_of_list(indices_of_list: dict, outfile_name: str):
    outfile_path = DATA_FOLDER / outfile_name
    with outfile_path.open('w', encoding='utf-8') as outfile:
        json.dump(indices_of_list, outfile)


indices_of_lists = get_indices_of_lists()
save_indices_of_list(indices_of_lists, 'BBR_FULL_indices_of_lists.json')


In [ ]:
# preprocessing_BBR

# %% Jupyter extensions
%load_ext autoreload
%autoreload 2

# %% Imports
from utils.preprocessing import get_file_metadata, json_list_to_csv

#%%
FullBuildingList_path = DATA_FOLDER / 'BuildingListBBR_FULL.json'
BuildingList_path = DATA_FOLDER / 'BBR_BuildingList.csv'

#%%
BBR_metadata_path = DATA_FOLDER / 'BBR_metadata.json'
BBR_metadata = get_file_metadata(BBR_DATA_PATH, BBR_metadata_path)
BBR_metadata

# %% HusnummerList
json_list_to_csv(
    BBR_DATA_PATH,
    BBR_metadata['BygningList'],
    BuildingList_path,
    attributes=[
        'id_lokalId',
        'status',
        'byg007Bygningsnummer',
        'byg021BygningensAnvendelse',
        'byg026Opførelsesår',
        'byg027OmTilbygningsår',
        'byg030Vandforsyning',
        'byg031Afløbsforhold',
        'byg032YdervæggensMateriale',
        'byg033Tagdækningsmateriale',
        'byg034SupplerendeYdervæggensMateriale',
        'byg035SupplerendeTagdækningsMateriale',
        'byg036AsbestholdigtMateriale',
        'byg038SamletBygningsareal',
        'byg039BygningensSamledeBoligAreal',
        'byg040BygningensSamledeErhvervsAreal',
        'byg041BebyggetAreal',
        'byg042ArealIndbyggetGarage',
        'byg043ArealIndbyggetCarport',
        'byg044ArealIndbyggetUdhus',
        'byg045ArealIndbyggetUdestueEllerLign',
        'byg046SamletArealAfLukkedeOverdækningerPåBygningen',
        'byg047ArealAfAffaldsrumITerrænniveau',
        'byg048AndetAreal',
        'byg049ArealAfOverdækketAreal',
        'byg051Adgangsareal',
        'byg054AntalEtager',
        'byg055AfvigendeEtager',
        'byg056Varmeinstallation',
        'byg057Opvarmningsmiddel',
        'byg058SupplerendeVarme',
        'byg069Sikringsrumpladser',
        'byg070Fredning',
        'byg111StormrådetsOversvømmelsesSelvrisiko',
        'byg112DatoForRegistreringFraStormrådet',
        'byg130ArealAfUdvendigEfterisolering',
        'byg136PlaceringPåSøterritorie',
        'jordstykke',
        'husnummer',
        'ejerlejlighed',
        'grund'
    ])

#%%
BygningList_data = pd.read_csv(BuildingList_path, delimiter=';')
# %%
columns_to_remove = []
for col in BygningList_data.columns:
    if not (~BygningList_data[col].isna()).any():
        print(col)
        columns_to_remove.append(col)

new_BygningList_data = BygningList_data.loc[:, set(BygningList_data.columns) - set(columns_to_remove)]

#%%
new_BygningList_data.to_csv(
    DATA_FOLDER / 'BBR_BuildingList2.csv', sep=';')

#%% Remove Apartments
BygningList_data_no_apartments = new_BygningList_data[new_BygningList_data['ejerlejlighed'].isna()]

#%% Remove non-residence buildings
residence_building_ids = [120, 121, 122, 130, 131, 132, 140, 190]
BygningList_data_residence_places = BygningList_data_no_apartments[BygningList_data_no_apartments['byg021BygningensAnvendelse'].isin(residence_building_ids)]
# %% Save the filtered version
BygningList_data_residence_places.to_csv(
    DATA_FOLDER / 'BBR_BuildingList_filtered.csv', sep=';')


### Danmarks Adresseregister (DAR)

In [ ]:
# preprocessing_DAR

# %% Jupyter extensions
%load_ext autoreload
%autoreload 2

from utils.preprocessing import get_file_metadata, json_list_to_csv

# %% Data paths
DAR_path = Path('D:/DAR_Aktuelt_Totaludtraek_JSON_HF_20231105180006/DAR_Aktuelt_Totaludtraek_JSON_HF_20231105180006.json')
HusnummerList_path = Path('data/DAR/DAR_HusnummerList.csv')
PostnummerList_path = Path('data/DAR/DAR_PostnummerList.csv')
NavngivenVejList_path = Path('data/DAR/DAR_NavngivenVejList.csv')
#############################################################
# Gathering data
#############################################################
# %% Generate Metadata
DAR_metadata_path = Path('data/DAR/DAR_metadata.json')
DAR_metadata = get_file_metadata(DAR_path, DAR_metadata_path)
DAR_metadata
########################## DAR data to csv ##########################
# %% HusnummerList
json_list_to_csv(
    DAR_path,
    DAR_metadata['HusnummerList'],
    HusnummerList_path,
    attributes=[
        'id_lokalId',
        'status',
        'adgangsadressebetegnelse',
        'husnummertekst',
        'navngivenVej',
        'postnummer'
    ])

# %% PostnummerList
json_list_to_csv(
    DAR_path,
    DAR_metadata['PostnummerList'],
    PostnummerList_path,
    attributes=[
        'id_lokalId',
        'status',
        'navn',
        'postnr'
    ])
# %% NavngivenVejList
json_list_to_csv(
    DAR_path,
    DAR_metadata['NavngivenVejList'],
    NavngivenVejList_path,
    attributes=[
        'id_lokalId',
        'status',
        'vejnavn'
    ])

########################## Join DAR files ##########################
# %% Loading DAR data
HusnummerList_data = pd.read_csv(HusnummerList_path, delimiter=';')\
                        .rename(columns={
                            'id_lokalId':'husnummer_id',
                            'adgangsadressebetegnelse': 'address',
                            'husnummertekst':'house_nr',
                            'navngivenVej':'street_id',
                            'postnummer':'postal_id'
                        })
NavngivenVejList_data = pd.read_csv(NavngivenVejList_path, delimiter=';')\
                        .rename(columns={
                            'id_lokalId':'street_id',
                            'vejnavn':'street_name'
                        })
PostnummerList_data = pd.read_csv(PostnummerList_path, delimiter=';')\
                        .rename(columns={
                            'id_lokalId':'postal_id',
                            'navn': 'city_name',
                            'postnr':'postal_nr'
                        })
# %% Filter addresses
has_street = ~pd.isna(HusnummerList_data['street_id'])
is_house = HusnummerList_data['house_nr'].apply(lambda x: str.isnumeric(str(x)))

DAR_data = HusnummerList_data.loc[has_street]
DAR_data
# %% Join DAR data
DAR_joined_data = DAR_data.merge(NavngivenVejList_data.loc[:,['street_id','street_name']], how='left', on='street_id')\
                    .merge(PostnummerList_data.loc[:, ['postal_id','city_name','postal_nr']], how='left', on='postal_id')\
                    .loc[:, ['husnummer_id','address','house_nr','street_name','city_name','postal_nr']]
DAR_joined_data
# %% Save DAR data
DAR_joined_data_path = Path('data/DAR/DAR_joined_full.csv')
DAR_joined_data.to_csv(DAR_joined_data_path, sep=';', index=False)
# %%


### Ejendomsvurderinger (VUR)

In [ ]:

# VUR_merge. Run from command line as independent script
    
"""
CL args:
    VUR_DIR (str): Directory of VUR files e.g "/home/downloads". Defaults to current working directory
"""
if sys.argv[1]:
    VUR_DIR = str(sys.argv[1])
else:
    VUR_DIR = "."

subset = ['id']

joined_files = os.path.join(VUR_DIR, "VUR_*.csv") 
joined_list = glob.glob(joined_files)
df = pd.concat(map(lambda file: pd.read_csv(file, sep=';'), joined_list), ignore_index=True)
print(df.head())
before = len(df)
mask = df.duplicated(subset=subset)

print(f"Before removing duplicates we have {before} rows")

df.drop_duplicates(subset=subset, inplace=True, ignore_index=True)
df.to_csv(os.path.join(VUR_DIR, "VUR_joined_with_nulls.csv"), index=False, sep=';')

print(f"After removing duplicates we have {len(df)} rows")
print(f"Removed {before - len(df)} rows in total") 
df.loc[mask].to_csv(os.path.join(VUR_DIR, "VUR_joined_duplicates.csv"), index=False, sep=';')

df_no_nulls = df.dropna(how='any')
df_no_nulls.to_csv(os.path.join(VUR_DIR, "VUR_joined.csv"), index=False, sep=';')
print("Dropped nulls df:", len(df_no_nulls))

### Attributes

In [ ]:
# Attribute overview
{
    "id": GUID # aka. adgangsAdresseID(VUR), husnummer_id(BBR,DAR),
    "postal_nr": str # DAR,
    "address": str # DAR,
    "evaluationYear": int # VUR,
    "propertyValue": float # VUR,
    "groundValue": float # VUR, 
    "propertyTax": float # VUR,
    "groundTax": float # VUR,
    "totalAddressTax": float # VUR,
    "buildingUsageType": one hot encoded # aka. byg021BygningensAnvendelse(BBR),
    "constructionYear": int # aka. byg026Opførelsesår(BBR),
    "reconstructionOrExtensionYear": int # byg027OmTilbygningsår(BBR),
    "waterSupplyType": one hot encoded # aka. byg030Vandforsyning(BBR),
    "drainageType": one hot encoded # aka. byg031Afløbsforhold(BBR),
    "outerWallMaterialType": one hot encoded # aka. byg032YdervæggensMateriale(BBR),
    "roofMaterialType": one hot encoded # aka. byg033Tagdækningsmateriale(BBR),
    "supplementOuterWallMaterialType": one hot encoded # aka. byg034SupplerendeYdervæggensMateriale(BBR) - code is YdervæggensMateriale,
    "supplementRoofMaterialType": one hot encoded # aka. byg035SupplerendeTagdækningsMateriale(BBR) - code is Tagdækningsmateriale,
    "asbestosHoldingMaterialType": one hot encoded # aka. byg036AsbestholdigtMateriale(BBR),
    "totalBuildingArea": float # aka. byg038SamletBygningsareal(BBR),
    "totalResidenceArea": float # aka. byg039BygningensSamledeBoligAreal(BBR),
    "totalIndustrialArea": float # aka. byg040BygningensSamledeErhvervsAreal(BBR),
    "builtArea": float # aka. byg041BebyggetAreal (BBR),
    "areaOfBuiltInGarage": float # aka. byg042ArealIndbyggetGarage (BBR),
    "areaOfBuiltInCarport": float # aka. byg043ArealIndbyggetCarport (BBR),
    "areaOfBuiltInShed": float # aka. byg044ArealIndbyggetUdhus (BBR),
    "areaOfBuiltInConservatoryOrSimilar": float # aka. byg045ArealIndbyggetUdestueEllerLign (BBR),
    "totalAreaOfClosedCoveringsOnTheBuilding": float # aka. byg046SamletArealAfLukkedeOverdækningerPåBygningen (BBR),
    "areaOfWasteRoomAtGroundLevel": float # aka. byg047ArealAfAffaldsrumITerrænniveau (BBR),
    "otherArea": float # aka. byg048AndetAreal (BBR),
    "areaOfCoveredArea": float # aka. byg049ArealAfOverdækketAreal (BBR),
    "accessArea": float # aka. byg051Adgangsareal (BBR),
    "numberOfFloors": int # aka. byg054AntalEtager (BBR),
    "deviantFloors": one hot encoded # aka. byg055AfvigendeEtager (BBR),
    "heatingInstallation": one hot encoded # aka. byg056Varmeinstallation (BBR),
    "heatingMedium": one hot encoded # aka. byg057Opvarmningsmiddel (BBR),
    "supplementaryHeating": one hot encoded # aka. byg058SupplerendeVarme (BBR) - code list is Opvarmningsmiddel,
    "shelterSpaces": int # aka. byg069Sikringsrumpladser (BBR),
    "preservation": one hot encoded # aka. byg070Fredning (BBR),
    "stormCouncilsFloodSelfRisk": one hot encoded # aka. byg111StormrådetsOversvømmelsesSelvrisiko (BBR) - code list is OversvømmelsesSelvrisiko,
    "areaOfExternalInsulation": float # aka. byg130ArealAfUdvendigEfterisolering (BBR),
    "locationOnLakeTerritory": bool # aka. byg136PlaceringPåSøterritorie (BBR),
    "totalAddressValue": float # sum of propertyValue and groundValue,
    "region": int # first digit of postal_nr
}

# Building dataset

In [ ]:
# preprocessing
# %% Settings
data_path = Path('data')
BBR_path = data_path / Path('BBR/BBR_BuildingList.csv')
VUR_path = data_path / Path('VUR/VUR_joined.csv')
DAR_path = data_path / Path('DAR/DAR_joined.csv')

# %% Load data
BBR = pd.read_csv(BBR_path, delimiter=';', index_col=0)
VUR = pd.read_csv(VUR_path, delimiter=';')\
    .drop(columns=['id'])\
    .rename(columns={'vurderingsaar':'evaluationYear', 'adgangsAdresseID':'id'})
DAR = pd.read_csv(DAR_path, delimiter=';')\
    .rename(columns={'husnummer_id':'id'})
# %% Rename BBR columns
DA_EN_BBR_column_map = {
    "byg021BygningensAnvendelse" : "buildingUsageType",
     "byg026Opførelsesår" : "constructionYear",
     "byg027OmTilbygningsår" : "reconstructionOrExtensionYear",
     "byg030Vandforsyning" : "waterSupplyType",
     "byg031Afløbsforhold" : "drainageType",
     "byg032YdervæggensMateriale" : "outerWallMaterialType",
     "byg033Tagdækningsmateriale" : "roofMaterialType",
     "byg034SupplerendeYdervæggensMateriale" : "supplementOuterWallMaterialType",
     "byg035SupplerendeTagdækningsMateriale" : "supplementRoofMaterialType",
     "byg036AsbestholdigtMateriale" : "asbestosHoldingMaterialType",
     "byg038SamletBygningsareal" : "totalBuildingArea",
     "byg039BygningensSamledeBoligAreal" : "totalResidenceArea",
     "byg040BygningensSamledeErhvervsAreal" : "totalIndustrialArea",
     "byg041BebyggetAreal"  : "builtArea",
     "byg042ArealIndbyggetGarage"  : "areaOfBuiltInGarage",
     "byg043ArealIndbyggetCarport"  : "areaOfBuiltInCarport",
     "byg044ArealIndbyggetUdhus"  : "areaOfBuiltInShed",
     "byg045ArealIndbyggetUdestueEllerLign"  : "areaOfBuiltInConservatoryOrSimilar",
     "byg046SamletArealAfLukkedeOverdækningerPåBygningen"  : "totalAreaOfClosedCoveringsOnTheBuilding",
     "byg047ArealAfAffaldsrumITerrænniveau"  : "areaOfWasteRoomAtGroundLevel",
     "byg048AndetAreal"  : "otherArea",
     "byg049ArealAfOverdækketAreal"  : "areaOfCoveredArea",
     "byg051Adgangsareal"  : "accessArea",
     "byg054AntalEtager"  : "numberOfFloors",
     "byg055AfvigendeEtager"  : "deviantFloors",
     "byg056Varmeinstallation"  : "heatingInstallation",
     "byg057Opvarmningsmiddel"  : "heatingMedium",
     "byg058SupplerendeVarme" : "supplementaryHeating",
     "byg069Sikringsrumpladser"  : "shelterSpaces",
     "byg070Fredning"  : "preservation",
     "byg111StormrådetsOversvømmelsesSelvrisiko": "stormCouncilsFloodSelfRisk",
     "byg130ArealAfUdvendigEfterisolering"  : "areaOfExternalInsulation",
    "byg136PlaceringPåSøterritorie"  :     "locationOnLakeTerritory",
    "husnummer": "id"
}
BBR.rename(columns=DA_EN_BBR_column_map, inplace=True)
BBR.drop(columns=set(BBR.columns) - set(DA_EN_BBR_column_map.values()), inplace=True)
BBR.columns
# %% Get the latest BBR data for a given address
BBR_filtered = BBR.groupby('id').max('constructionYear')
# %% Merge data
ds = VUR.merge(DAR, how='left', on='id')\
        .merge(BBR_filtered, how='inner', on='id')

ds
# %% Save dataset
ds.to_csv('data/DS.csv',sep=';', index=False)

# %% One hot encode categorical attributes
categorical_columns = [
    'buildingUsageType',
    'waterSupplyType',
    'drainageType',
    'outerWallMaterialType',
    'roofMaterialType',
    'supplementOuterWallMaterialType',
    'supplementRoofMaterialType',
    'asbestosHoldingMaterialType',
    'deviantFloors',
    'heatingInstallation',
    'heatingMedium',
    'supplementaryHeating',
    'preservation',
    'stormCouncilsFloodSelfRisk'
]
one_hot_ds = pd.get_dummies(ds, columns=categorical_columns, dtype=int)
# %% Save one hot ds
one_hot_ds.to_csv('data/DS_with_one_hot.csv',sep=';', index=False)

# Clustering

### K-means

In [ ]:
# Kmeans_hyperparameter
# %% Load data
ds_path = 'data/DS_RAndD_minmax.csv'
print('[Loading data] Start:', ds_path)
ds = pd.read_csv(ds_path, index_col='id')
print('[Loading data] Done')
# %% Separate data
feature_names = ds.columns[ds.columns != PREDICTION_VAR]
feature_data = ds[feature_names]

# %% Determine optimal K
sum_of_sq_dists = []
Ks = range(100,300,25)
for K in Ks:
    kmeans = KMeans(n_clusters=K, random_state=42, n_init=10)
    kmeans.fit(feature_data)
    sum_of_sq_dists.append(kmeans.inertia_)
plt.plot(Ks,sum_of_sq_dists, '.-')
plt.xlabel('K')
plt.ylabel('Sum of squared distances')
plt.title('Optimal K')
plt.savefig('../assets/Kmeans.png')

In [ ]:
# Kmeans_clustering
# %% Load data
print('[Loading data] Start')
ds = pd.read_csv('data/DS_RAndD_minmax.csv', index_col='id')
print('[Loading data] Done')
# %% Separate data
feature_names = ds.columns[ds.columns != PREDICTION_VAR]
feature_data = ds[feature_names]
# %% Hyperparameters
K = 250

# %% Fit clusters
print('[Determining clusters] Start')
kmeans = KMeans(n_clusters=K)
labels = kmeans.fit_predict(feature_data)
print('[Determining clusters] Done')
# %% Save clusters
np.save(f"data/clusterings/Kmeans_clusterings_K{str(K)}.npy", labels)

### DBSCAN

In [ ]:
# DBSCAN_hyperparameter_analysis
# %% Load data
ds_path = 'data/DS_reduced_and_dropped.csv'
# ds_path = 'data/DS_RAndD_minmax.csv'
# ds_path = 'data/DS_RAndD_normalized.csv'
print('[Loading data] Start:', ds_path)
ds = pd.read_csv(ds_path, index_col='id')
print('[Loading data] Done')
# %% Separate data
feature_names = ds.columns[ds.columns != PREDICTION_VAR]
feature_data = ds[feature_names]

# %% Determine distances
NN_model = NearestNeighbors(n_neighbors=2, n_jobs=-1).fit(feature_data)
distances, indices = NN_model.kneighbors(feature_data)

# %% Save distances
np.save('data/distances/DBSCAN_HP_distances.npy',distances)
# np.save('data/distances/DBSCAN_HP_distances_minmax.npy',distances)
# np.save('data/distances/DBSCAN_HP_distances_normalized.npy',distances)

In [ ]:
# DBSCAN_clustering
# %% Load data
print('[Loading data] Start')
ds = pd.read_csv('data/DS_RAndD_minmax.csv', index_col='id')
print('[Loading data] Done')
# %% Separate data
feature_names = ds.columns[ds.columns != PREDICTION_VAR]
feature_data = ds[feature_names]
# %% Hyperparameters
# https://medium.com/@tarammullin/dbscan-parameter-estimation-ff8330e3a3bd
eps = 1.1
min_samples = 2*len(feature_names)

# %% Fit clusters
print('[Determining clusters] Start')
dbscan = DBSCAN(eps=eps, min_samples=min_samples, n_jobs=-1)
dbscan.fit(feature_data)
print('[Determining clusters] Done')
# %% Save clusters
db_clusterings = dbscan.labels_
np.save(f"data/clusterings/DBSCAN_clusterings_minmax_eps{str(eps).replace('.','_')}_ms{min_samples}.npy", db_clusterings)

was run on HPC using ```queue_run.sh```

In [ ]:
#!/bin/sh
#BSUB -q hpc
#BSUB -J DBSCAN
#BSUB -n 4
#BSUB -W 3:00
#BSUB -R "rusage[mem=64GB]"
#BSUB -o out/JLOG_%J.out
#BSUB -N 

echo '=================== Load modules: Started ==================='
module load python3
#module load scipy/1.10.1-python-3.11.3
#nvidia-smi
#module load cuda/12.1
echo '=================== Load modules: Succeded ==================='

echo '=================== Activate environment: Start ==================='
source 02807/bin/activate
echo '=================== Activate environment: Succeded ==================='

echo '=================== Executing script: Start ==================='
python3 analysis/DBSCAN_clustering.py
echo '=================== Executing script: Succeded ===================

## (Polynomial regression)
We found out this did not work too well

In [ ]:
# %% Load data
print('[Loading data] Start')
ds = pd.read_csv('data/DS_reduced_and_dropped.csv', index_col='id')
print('[Loading data] Done')
# %% Separate data
# Prediction variable
prediction_property = 'totalAddressValue'
total_address_values = ds[prediction_property]

# Feature data
feature_names = ds.columns[ds.columns != prediction_property]
feature_data = ds[feature_names]
#############################################################
# %% No scaling
#############################################################
print('[No scaling] Fitting')
poly_features = PolynomialFeatures(degree=2, include_bias=False)\
    .fit_transform(feature_data)
poly_model = LinearRegression(n_jobs=-1).fit(poly_features, total_address_values)
print('[No scaling] Fitting done')

# Coefficient of determination
coef_det = poly_model.score(poly_features, total_address_values)
print(f'[No scaling] Coefficient of determination: {coef_det}')

if coef_det > 0.315:
    with open('data/models/PR_model.sav', 'wb') as f:
        pickle.dump(poly_model, f)
#############################################################
# %% Standard scaling
#############################################################
scaler = StandardScaler()
normalized_feature_data = scaler.fit_transform(feature_data)

print('[Standard scaling] Fitting')
poly_features = PolynomialFeatures(degree=2, include_bias=False)\
    .fit_transform(normalized_feature_data)
poly_model = LinearRegression(n_jobs=-1).fit(poly_features, total_address_values)
print('[Standard scaling] Fitting done')

# Coefficient of determination
coef_det = poly_model.score(poly_features, total_address_values)
print(f'[Standard scaling] Coefficient of determination: {coef_det}')

if coef_det > 0.315:
    with open('data/models/PR_standard_model.sav', 'wb') as f:
        pickle.dump(poly_model, f)
#############################################################
# %% MinMax scaling
#############################################################
scaler = MinMaxScaler()
normalized_feature_data = scaler.fit_transform(feature_data)

print('[MinMax scaling] Fitting')
poly_features = PolynomialFeatures(degree=2, include_bias=False)\
    .fit_transform(normalized_feature_data)
poly_model = LinearRegression(n_jobs=-1).fit(poly_features, total_address_values)
print('[MinMax scaling] Fitting done')

# Coefficient of determination
coef_det = poly_model.score(poly_features, total_address_values)
print(f'[MinMax scaling] Coefficient of determination: {coef_det}')

if coef_det > 0.315:
    with open('data/models/PR_minmax_model.sav', 'wb') as f:
        pickle.dump(poly_model, f)

# Abnormality detection


In [ ]:

tqdm.pandas()
warnings.filterwarnings("ignore")

# %% Load data
sys.stdout.write('[Loading data] Start\n')
ds = pd.read_csv('data/DS_RAndD_minmax.csv', index_col='id')

evaluations = ds[PREDICTION_VAR]
feature_names = ds.columns[ds.columns != PREDICTION_VAR]
feature_data = ds[feature_names]
sys.stdout.write('[Loading data] Done\n')
# %% Load clusters
sys.stdout.write('[Loading clusterings] Start\n')
clusterings = np.load('data/clusterings/Kmeans_clusterings_K250.npy')
sys.stdout.write('[Loading clusterings] Done\n')
# %% Hyperparameters
K = 100
T = 1/100
reference_dist = norm()
# %% Abnormality analysis
def check_outlier_abnormality(row, NN_model, evaluations):
    distances, indicies = NN_model.kneighbors(row[feature_names].to_numpy().reshape(1, -1))
    NN_evaluations = evaluations[indicies[0][1:]]
    n = len(NN_evaluations)
    tobs = (row[PREDICTION_VAR] - NN_evaluations.mean())/(NN_evaluations.std()/np.sqrt(n))
    p = 2*(1 - reference_dist.cdf(abs(tobs)))

    return 0 if p > T else 1

def check_abnormality(row, evaluations):
    evaluations = evaluations[evaluations.index != row.name]
    tobs = (row[PREDICTION_VAR] - evaluations.mean())/(evaluations.std())
    p = (1 - reference_dist.cdf(abs(tobs)))

    return 0 if p > T else 1

abnormality_ds = pd.DataFrame()
cluster_ids = np.unique(clusterings)
for cluster_id in tqdm(cluster_ids):
    cluster_mask = clusterings==cluster_id
    cluster = ds[cluster_mask]
    cluster['cluster'] = cluster_id

    if cluster_id == -1: # Outliers
        cluster['isAbnormal'] = 1
        # cluster_NN = NearestNeighbors(n_neighbors=K+1).fit(feature_data) # K+1 because it will find itself
        # cluster['isAbnormal'] = cluster.progress_apply(lambda row: check_outlier_abnormality(row, cluster_NN, evaluations), axis=1)
    else:
        cluster['isAbnormal'] = cluster.progress_apply(lambda row: check_abnormality(row, cluster[PREDICTION_VAR]), axis=1)

    abnormality_ds = pd.concat([abnormality_ds, cluster])
# %% Save abnormality
abnormality_ds.to_csv('data/DS_abnormality.csv')
# %%
stds = []
for cluster_id in tqdm(cluster_ids):
    cluster_mask = clusterings==cluster_id
    cluster = ds[cluster_mask]
    
    stds.append(cluster[PREDICTION_VAR].std())

stds
# %% Check abnormal valuations
abnormalities = abnormality_ds.loc[(abnormality_ds['isAbnormal'] == 1) & (abnormality_ds['cluster'] != -1)]
DS = pd.read_csv('/data/DS.csv', sep=';', index_col='id')
DS_sub = DS[list(set(DS.columns) - set(abnormalities.columns))]
# %%
abnorm_ds = abnormalities.merge(DS_sub, how='inner', on='id')
abnorm_ds[['address','propertyValue','groundValue']]
# %% Normal
normal = abnormality_ds.loc[abnormality_ds['isAbnormal'] == 0]
normal_ds = normal.merge(DS_sub, how='inner', on='id')
normal_ds[['address','propertyValue','groundValue']]

# Evaluation

In [ ]:
# %% Imports
tqdm.pandas()
warnings.filterwarnings("ignore")
# %% Load data
print('[Loading data] Start')
ds = pd.read_csv('../data/DS_abnormality.csv', index_col='id')
print('[Loading data] Done')
# %% Configuration
feature_names = ds.columns[ds.columns != PREDICTION_VAR]
seed = 42
T = 1/100
reference_dist = norm()
multipliers = [1/2,2]
N = 1000
K = 5
############################################################
# Creating test dataset
############################################################
# %% Get normal obs and change their evaluation
normal_obs = ds.loc[ds['isAbnormal'] == 0].sample(n=N//2, random_state=seed).copy()
normal_obs['multiplier'] = np.random.choice(multipliers, len(normal_obs), replace=True)
normal_obs[PREDICTION_VAR] = normal_obs[PREDICTION_VAR]*normal_obs['multiplier']
normal_obs['label'] = 1
normal_obs

# %% Get abnormal ones and change evaluation to the average of the K nearest normal ones to check if it will make it normal
def get_average_of_normal_KNN(abnormal_row, NN_model: NearestNeighbors, evaluations: pd.Series):
    distances, indicies = NN_model.kneighbors(abnormal_row[feature_names].to_numpy().reshape(1, -1))
    NN_evaluations = evaluations[indicies[0][1:]]
    
    return NN_evaluations.mean()

abnormal_obs = ds.loc[(ds['isAbnormal'] == 1) & (ds['cluster'] != -1)].sample(n=N//2, random_state=seed).copy()
for cluster_id in tqdm(abnormal_obs['cluster'].unique()):
    norm_cluster = ds.loc[(ds['cluster'] == cluster_id) & (ds['isAbnormal'] == 0)]
    abnorm_mask = abnormal_obs['cluster'] == cluster_id
    abnorm_cluster = abnormal_obs.loc[abnorm_mask]

    cluster_NN = NearestNeighbors(n_neighbors=K).fit(norm_cluster[feature_names])
    abnormal_obs.loc[abnorm_mask, PREDICTION_VAR] = abnorm_cluster.apply(lambda row: get_average_of_normal_KNN(row, cluster_NN, norm_cluster[PREDICTION_VAR]), axis=1)

abnormal_obs['label'] = 0
abnormal_obs
# %% Test set
ds_test = pd.concat([normal_obs, abnormal_obs])
ds_test

############################################################
# Creating test dataset
############################################################
# %% Predict
for cluster_id in tqdm(ds_test['cluster'].unique()):
    cluster = ds.loc[ds['cluster'] == cluster_id]
    cluster_evals = cluster[PREDICTION_VAR]

    test_cluster_mask = ds_test['cluster'] == cluster_id
    test_cluster = ds_test.loc[test_cluster_mask]
    zs = (test_cluster[PREDICTION_VAR] - cluster_evals.mean())/(cluster_evals.std())
    ps = (1 - reference_dist.cdf(np.abs(zs)))

    ds_test.loc[test_cluster_mask,'prediction'] = (ps<T).astype(int)

ds_test

# %% Evaluation
print(confusion_matrix(ds_test['label'], ds_test['prediction'], normalize='true'))
(ds_test['label'] == ds_test['prediction']).mean()
# %%
